In [23]:
from openai import OpenAI
from pathlib import Path
import pinecone as pc
from uuid import uuid4
from pinecone import Pinecone as pc, ServerlessSpec
import openai
import re



In [24]:
pc = pc(api_key="placeholder")
try:
    #Create Pincone Index
    pc.create_index(
                    #Using the name of the Index
                    name="petsittingknowledge",
                    #With model dimension
                    dimension=1536,
                    #With cosine metric
                    metric="cosine",
                    #Wtih customized spec
                    spec=ServerlessSpec(
                                        cloud="aws",
                                        region="us-east-1"
                                        ) 
                    )
except:
    print("No need, Index Already Exists")
#Initialize Pinecone Index Object
index = pc.Index("petsittingknowledge") 

In [26]:
#Load Knowledge Base
with open("/Users/Jerry/Desktop/Rover/Assistant/j_sit_and_stay_knowledge.txt", "r") as f:
    text = f.read()

#Section-Based Chunking
pattern = r"(===.*?===)"
sections = re.split(pattern, text)

chunks = []
for i in range(1, len(sections), 2):
    title = sections[i].strip("= \n")
    content = sections[i + 1].strip()
    combined = f"{title}\n{content}"
    chunks.append((title, combined))

print(f"Split into {len(chunks)} sections.")

#Embedding + Upsert
vectors = []
for title, chunk in chunks:
    if chunk.strip():
        # Call embedding
        response = client.embeddings.create(
            input=chunk,
            model="text-embedding-ada-002"
        )
        vector = response.data[0].embedding

        # Add metadata only for pricing
        metadata = {"section": title, "text": chunk.strip()}
        if "Pricing" in title:
            metadata["type"] = "pricing"

        vectors.append({
            "id": str(uuid4()),
            "values": vector,
            "metadata": metadata
        })

#Upsert data and vectors into Pinecone
index.upsert(vectors=vectors)
print(f"Successfully upserted {len(vectors)} sections to Pinecone with metadata.")

✅ Split into 8 sections.
✅ Successfully upserted 8 sections to Pinecone with metadata.


In [37]:
query = "How much is dog's rate"
query_embedding = client.embeddings.create(input=query, model="text-embedding-ada-002").data[0].embedding
results = index.query(vector=query_embedding, top_k=1, include_metadata=True)

for match in results["matches"]:
    print(match["metadata"]["text"])

Pricing Information
*All pricing is night-based. Holiday rate is applied throughout the stay.*

Cat Rates:
- First Timer, Regular: $50
- First Timer, Holiday: $55
- Returning, Regular: $40
- Returning, Holiday: $45

Dog Rates:
- First Timer, Regular: $60
- First Timer, Holiday: $65
- Returning, Regular: $50
- Returning, Holiday: $55
